<a href="https://colab.research.google.com/github/jarrydmartinx/deep-rl/blob/master/distributional_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A Distributional Perspective on Reinforcement Learning (Bellemare, 2017)

Stems from the realisation cthat under the RL problem, the distribution over returns is very complex, e.g. may be multimodal. If you look at the mean, you're often looking at a reward you will *never* see in reality. The value (expected return) therefore hides the intrinsic randomness, and so we are not modelling an aspect of reality. This work is basically about *removing the expectations from the Bellman equation*.

### Concepts: Return, The Value, and CNC
* Value is expected sum of future rewards: $$V^\pi(x) = \mathbb{E}[Z^\pi(x)]  = \mathbb{E}\bigg[\sum_{t=0}^\infty R(x_t)\bigg]$$
* The return (sum of future reward) from a state $x$ is a random variable $Z^\pi$. 
* That means we can estimate it using Monte Carlo estimation
* CNC: Let's assume the reward is undiscounted and the horizon is finite. Then we have a tractable supervised learning problem. We have a number of possible outcomes (return amounts) and we can just predict all of them (learn a distribution over them)

####Bellman equations for different moments
* Bellman (1957): Bellman equation for mean (describes the relation between the mean return at one state and the mean return at another state)
$$ V^\pi(x) = \mathbb{E}\big[Z^\pi(x)\big] = \mathbb{E}R(x) + \gamma \mathbb{E}\big[Z^\pi(X')\big]$$
* But there are other Bellman equations for higher moments:
	* Sobel (1982): Bellman equation for variance
	* Engel (2003): Bellman equation for Bayesian uncertainty - this describes *our* uncertainty about the system, not the actual stochasticity of the return.
	* Azar et al. (2011), Lattimore & Hutter (2012): Bellman equation for higher moments.
* Bellemare is trying to quantify the *whole* distribution and not just certain moments, and hopefully that gives a more unified picture.

### Fixed Point of the Distributional Bellman Equation:
$$ Z^\pi(x) \overset{D}{=} R(x) + \gamma Z^\pi(X')\qquad X' \sim P^\pi(\cdot\mid x)$$
* This is an instance of a *recursive distributional equation*. They're also called *stochastic fixed point equations*. Here's a recursive distributional equation whose solution is the standard normal distribution $\mathcal{N}(0,1)$ (Rosler, 1992):
	$$ X \overset{D}{=}  \frac{1}{\sqrt{2}}X_1 + \frac{1}{\sqrt{2}}X_2 $$
	$$ X \sim X_1 \sim X_2$$

#### Look for an operator
Whenever B writes an RL paper he asks "Is there an **operator** I can look at?"
* An operator is basically a way of looking at the expected (average) behaviour of a learning algorithm
* When we do RL we sample states from the environment and we update. The operator tells you what happens to your current prediction after one update.
* So we have this value distribution which satisfies this recursive equation. So we have a fixed point...? He says this like it's obvious
* Now we ask: *is there an operator we could apply and use to study the learning behaviour*
* Let's call the operator $\mathcal{T}^\pi$ 

#### Applying the operator
The result of applying the operator to a given value distribution is going to be distributed like: $$ \mathcal{T}^\pi Z(x,a) :\overset{D}{=} R(x,a) + \gamma Z(X', A') $$
$$ X' \sim P(x\mid x,a)\qquad A' \sim \pi(\cdot\mid X') $$
So we have three sources of intrinsic randomness that define the compound distribution $\mathcal{T}^\pi Z$. This makes it fundamentally different to the usual Bellman Operator:
1. Reward
2. Next state
3. Next state return

## Theoretical Results
### The Distributional Bellman Operator $\mathcal{T}^\pi$ is a contraction in $\bar{d}_p$
##### Aside: The inverse c.d.f transform
 * The probability integral transform states that if $X$ is a continuous random variable with cumulative distribution function $F_{X}$ then the random variable $Y=F_{X}(X)$ has a uniform distribution on $[0, 1]$. 
 * The inverse probability integral transform is just the inverse of this: specifically, if  $Y$ has a uniform distribution on $[0, 1]$ and if $X$ has a cumulative distribution $F_{X}$ then the random variable $F_{X}^{-1}(Y)$ has the same distribution as $X$.

#### The Wasserstein Metric (Earth Mover Distance)
The Wasserstein metric measures the distance between two c.d.f's $F$ and $G$.
$$d_p(F,G) := \inf_{U,V}\left\Vert U - V\right\Vert $$
where the infimum is taken over all pairs of random variables $(U,V)$ with respective cumulative distributions F and G. The infimum is attained by the inverse c.d.f. transform of a random varible $\mathcal{U}$ that is uniformly distributed on $[0,1]$:
\begin{aligned}
 d_p(F,G)\
 :=& \left\Vert F^{-1}(\mathcal{U})-G^{-1}(\mathcal{U}) \right\Vert_p \\
 :=& \bigg( \int_0^1\left\vert F^{-1}(u) - G^{-1}(u)\right\vert^p du\bigg)^{\frac{1}{p}}
 \end{aligned}
* The last equation requires that $p<\infty$. I'm pretty sure the above is a result that allows you to compute the Wasserstein metric more easily (as opposed to somehow taking the infimum over "all pairs of RVs with cdfs $F$ and $G$").
* Note: Though inaccurate, they use $d_p(F,G)$  (W distance between cdfs) and $d_p(U,V)$ (where $U$ and $V$ are random variables) interchangeably, even though the W distance involves taking the inf over (all pairs of RVs with said c.d.f's) $U$ and $V$.

#### Maximal form of the Wasserstein metric, applied to returns
* Note that a value distribution $Z^\pi$ is a vector of random variables, with each element $Z^\pi(x,a)$ being a random variable corresponding to the distribution over returns for one state-action pair.
	* In other words it's a mapping from state-action pairs to distributions over returns.
* As before, let $\mathcal{U}$ be a random variable uniformly distributed on $[0,1]$. A maximal (over $\mathcal{X}\times\mathcal{A}$) form of the W metric is:
\begin{aligned}
\bar{d}_p(Z_1,Z_2) 
:=& \sup_{x,a} d_p(Z_1(x,a),Z_2(x,a)) \\
:=& \sup_{x,a}\inf_{Z_1(x,a),\ Z_2(x,a)} \left\Vert Z_1(x,a) - Z_2(x,a) \right\Vert \\
:=& \sup_{x,a}\left\Vert F_{Z_1(x,a)}^{-1}(\mathcal{U})-F_{Z_2(x,a)}^{-1}(\mathcal{U}) \right\Vert_p \\
 :=& \sup_{x,a}\bigg( \int_0^1\left\vert F_{Z_1(x,a)}^{-1}(u)-F_{Z_2(x,a)}^{-1}(u)\right\vert^p du\bigg)^{\frac{1}{p}}
\end{aligned}


#### Lemma 3: $\mathcal{T}^\pi$ is a contraction in $\bar{d}_p$
* Let $\mathcal{Z}$ denote the space of value distributions with bounded moments. For two value distributions $Z_1, Z_2\in\mathcal{Z}$, we make use of a maximal form of the Wasserstein metric (above). Now, consider the process $Z_{k+1} := \mathcal{T}^\pi Z_k$, starting with some $Z_0\in\mathcal{Z}$.
* We can expect the limiting expectation of $\{Z_k\}$ to converge exponentially quickly, as usual, to $Q^\pi$. But we want stronger convergence.

#### The argument for *distributional* convergence:
> 1. $\mathcal{T}^\pi$ is a contraction in $\bar{d}_p$, so all moments converge exponentially quickly.
2. By Banach's fixed point theorem, $\mathcal{T^\pi}$ has a unique fixed point.
3. By inspection (of the standard Q function def?) this fixed point must be $Z^\pi$.
4. Since we assumed all moments were bounded, this is sufficient to conclude that the sequence $\{Z_k\}$ converges to $Z^\pi$ in $\bar{d}_p$ for $1\leq p \leq \infty$.

Note that it is not a contraction in total variation distance, KL-divergence, or Kolmogorov distance.

## Algorithms
### The categorical algorithm and Categorical DQN
They propose an algorithm based on the distributional Bellman operator.
#### Model: Parametric approximation to the value distribution
We model the value distribution $Z$ it using a discrete distribution $Z_\theta$ parametrized by
* $N\in\mathbb{N}$ the number of canonical returns (return buckets)
* $V_{min},V_{max}\in\mathbb{R}$

Its support is the set of atoms $\{z_i = V_{min} + i\Delta z : 0 \leq i < N\}, \Delta z := \frac{V_{max}-V_{min}}{N-1}$. This discrete distribution is computationally friendly and expressive (apparently says so in the PixelRNN paper).
#### Loss
* The analysis in section 3 suggests that the Wasserstein metric would be a natural distance to minimize as a loss between $\mathcal{T}Z_\theta$ and $Z_\theta$.
* It is also robust to discrepancies in support, which we need. BUT it doesn't work for some other reason, because we're learning from sample transitions, which isn't possible under the Wasserstein loss.
* Instead they project the sample Bellman update $\hat{\mathcal{T}}Z_\theta$ onto the support of $Z_\theta$
* The sample loss is then the KL-divergence between the approximate value distribution and the projected sample Bellman update (equation 7)
#### Learning algorithm
A modified DQN architecture that outputs the atom probabilities $p_i(x,a)$ instead of action-values. 
* It's pretty natural to use DQN, instead of outputting a 1D vector of values for each action (given a state), you output a distribution over return for each action, given a state: i.e. a 2D tensor of return-bucket-probabilities (the elements of the value distribution) $\times$ actions.
* **Exploration**: Simple $\epsilon$-greedy policy. THey leave to future work the many ways in which an agent could select actions on the basis of the full distribution.

## Results
Why does it work better?
* Results suggest that value distributions are better able to propagate rarely occurring events. 
	* Makes sense because rarely occuring events with high reward might have that high reward averaged out if the mean of the return is always taken.
* Also works much better in the stochastic setting (random action rejection by environment)

